In [3]:
import pandas as pd
import psycopg2
from io import StringIO

In [4]:
# read in csv and check data
df = pd.read_csv('marvels_coupons.csv')
df.head(10)

In [60]:
# coupon codes are in four different columns append them into one series
codes = pd.Series()
for column in df:
    codes = codes.append(df[column])
# Add 'BM' string at the beginning of each code
codes = codes.map('BM{}'.format)

In [59]:
# Move codes into dataframe with discount % (25%) and discount type (1 for each of these)
df = pd.DataFrame(list(zip(codes,[25]*len(codes),[1]*len(codes))))

In [58]:
postgres database parameters
params = {
    "host"      : "localhost",
    "database"  : "db",
    "user"      : "user",
    "password"  : "password",
    "port"      :  5432
}

def connect_db(params):
    # connect to server
    conn = None
    try:
        print('Connecting...')
        conn = psycopg2.connect(**params)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print('Connected')
    return conn

def copy_df(conn, df, table):
    # initialize buffer
    buffer = StringIO()
    # copy dataframe to buffer in memory
    df.to_csv(buffer, index=False, header=False)
    buffer.seek(0)

    cur = conn.cursor()
    try:
        cur.copy_from(buffer, table, sep=',')
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print('Error: {}'.format(error))
        conn.rollback()
        return 1
    print('Copy Complete')
    cur.close()

# connect to db
conn = connect_db(params)
# copy in dataframe
copy_df(conn, df, 'coupons')
# close connect to db
conn.close()

Connecting...
Connected
Copy Complete
